## Libraries

In [3]:
# Data Processing
import numpy as np
import pandas as pd

# Calendar Information
import datetime
import holidays
import calendar

# Plotting
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns


# Data Preprocessing
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import MinMaxScaler

# Modeling and Forecasting
import optuna
from lightgbm import LGBMRegressor, early_stopping, log_evaluation
from tensorflow.keras import models, layers, callbacks, Input

# Evaluation
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import root_mean_squared_error
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.stats.diagnostic import acorr_ljungbox

# Other
import os
import joblib
import time


In [43]:
# Base path for saving plots
base_path_plots = "/Users/Vageli/Desktop/DABN01/plots/"

## Load data

### Energy Output

In [4]:
# Load energy output data
output = pd.read_csv(
    '/Users/Vageli/Desktop/DABN01/data/energy_quant_sols1_data/measurements.csv',
    sep=';',
    decimal=','
)

# Set index and name it 'datetime'
output.index = pd.to_datetime(output['Index'], format='%d.%m.%Y %H:%M')
output.index.name = 'datetime'

# Drop the original 'Index' column
output.drop(columns=['Index'], inplace=True)

# Rename the first column to 'output'
output.rename(columns={output.columns[0]: 'output'}, inplace=True)

# Check timezone info
print(output.index.tz)

# Convert to CET and handle DST properly
output_dst = output.tz_localize('CET', ambiguous='infer')


None


### Temperature

In [5]:
# Load temperature data
temperature = pd.read_csv(
    '/Users/Vageli/Desktop/DABN01/data/energy_quant_sols1_data/temperature_stuttgart_h.csv',
    sep=',',
    decimal='.',
    index_col=0,
    parse_dates=True
)

# Rename index and first column
temperature.index.name = 'datetime'
temperature.rename(columns={temperature.columns[0]: 'temperature'}, inplace=True)

# Check timezone info
print(temperature.index.tz)

# Convert to CET
temperature_dst = temperature.tz_convert('CET')

# Upsample to quarter-hourly frequency and forward fill
temperature_dst = temperature_dst.resample('15min').ffill()

UTC+01:00


### Merge Into Single DataFrame

In [6]:
# Merge into a single DataFrame
data = output_dst.join(temperature_dst, how='left')

# Drop NaNs
data = data.dropna()

# Check dates in merged dataset
print(data.index.min())
print(data.index.max())

# Print data length
print(f'Number of observations: {len(data)}')

2020-01-01 00:00:00+01:00
2024-12-16 23:45:00+01:00
Number of observations: 173952


In [7]:
# Show missing values per column
data.info()
data = data.dropna()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 173952 entries, 2020-01-01 00:00:00+01:00 to 2024-12-16 23:45:00+01:00
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   output       173952 non-null  float64
 1   temperature  173952 non-null  float64
dtypes: float64(2)
memory usage: 4.0 MB


## Feature Engineering

### General Features

#### Holiday Encoding

In [33]:
# Create a set of German holidays (province:'BW' for Baden-Württemberg)
de_holidays = holidays.Germany(prov='BW', years=range(data.index.year.min(), data.index.year.max() + 1))

# Encode as 1 if the date is a national holiday
data['is_holiday'] = data.index.map(lambda x: 1 if x.date() in de_holidays else 0)

#### Cyclical Encoding

In [34]:
# Extract temporal information
data['hour'] = data.index.hour           # 0-23
data['dayofweek'] = data.index.dayofweek  # 0-6
data['month'] = data.index.month          # 1-12

# Define cyclical transformers
def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(2 * np.pi * x / period))

def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(2 * np.pi * x / period))

# Apply cyclical encoding
data['hour_sin'] = sin_transformer(24).fit_transform(data[['hour']])
data['hour_cos'] = cos_transformer(24).fit_transform(data[['hour']])

data['dayofweek_sin'] = sin_transformer(7).fit_transform(data[['dayofweek']])
data['dayofweek_cos'] = cos_transformer(7).fit_transform(data[['dayofweek']])

data['month_sin'] = sin_transformer(12).fit_transform(data[['month']])
data['month_cos'] = cos_transformer(12).fit_transform(data[['month']])

### LightGBM Features

#### Lags of Response Variable

In [35]:
# Create lag features
data['lag_1w'] = data['output'].shift(672)     # 1 week
data['lag_1m'] = data['output'].shift(2880)    # 1 month
data['lag_1y'] = data['output'].shift(35040)   # 1 year

#### Rolling Windows of Temperature

In [36]:
# Create rolling window features for temperature
data['temp_roll_mean_1_day'] = data['temperature'].rolling(window=96).mean()
data['temp_roll_mean_7_day'] = data['temperature'].rolling(window=96 * 7).mean()

data['temp_roll_max_1_day'] = data['temperature'].rolling(window=96).max()
data['temp_roll_min_1_day'] = data['temperature'].rolling(window=96).min()

data['temp_roll_max_7_day'] = data['temperature'].rolling(window=96 * 7).max()
data['temp_roll_min_7_day'] = data['temperature'].rolling(window=96 * 7).min()

In [37]:
# Drop rows with NaNs caused by shifting
data.dropna(inplace=True)

### Data Info

In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 138912 entries, 2020-12-31 00:00:00+01:00 to 2024-12-16 23:45:00+01:00
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   output                138912 non-null  float64
 1   temperature           138912 non-null  float64
 2   is_holiday            138912 non-null  int64  
 3   hour                  138912 non-null  int32  
 4   dayofweek             138912 non-null  int32  
 5   month                 138912 non-null  int32  
 6   hour_sin              138912 non-null  float64
 7   hour_cos              138912 non-null  float64
 8   dayofweek_sin         138912 non-null  float64
 9   dayofweek_cos         138912 non-null  float64
 10  month_sin             138912 non-null  float64
 11  month_cos             138912 non-null  float64
 12  lag_1w                138912 non-null  float64
 13  lag_1m                138912 non-null  float64
 14  lag_1y

## General Funcitons

### Evaluation Metrics

In [44]:
# Create funciton that calculates and prints evaluation metrics
def evaluate_regression_metrics(y_true, y_pred, label="Model"):
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    rmse = root_mean_squared_error(y_true, y_pred)
    
    print(f"--- {label} Evaluation ---")
    print(f"Mean Absolute Error (MAE): {mae:.2f} kW")
    print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f} %")
    print(f"Root Mean Squared Error (RMSE): {rmse:.2f} kW")
    
    return {"MAE": mae, "MAPE": mape, "RMSE": rmse}

### Interactive Plot (Prediction vs Actual)

In [45]:
def plot_predictions(y_true, y_pred, dates=None, title="Prediction vs Actual Output", label="Model"):
    fig = go.Figure()
    x_axis = dates if dates is not None else list(range(len(y_true)))

    fig.add_trace(go.Scatter(x=x_axis, y=y_true, mode='lines', name='Actual Output'))
    fig.add_trace(go.Scatter(x=x_axis, y=y_pred, mode='lines', name=f'Predicted Output ({label})'))

    fig.update_layout(
        title=title,
        xaxis_title="Time" if dates is not None else "Time Step (15-min intervals)",
        yaxis_title="Output (kW)",
        legend=dict(x=0, y=1.1, orientation='h'),
        height=400,
        width=900,
        margin=dict(l=40, r=40, t=40, b=40)
    )

    fig.show()

### Plot (Prediction vs Actual)

In [46]:
def plot_and_save_predictions(datetime_series, y_true, y_pred, 
                              filename, 
                              base_path=base_path_plots,
                              xlabel="Date", ylabel="Energy Output (kW)"):
    
    # Full save path
    save_path = f"{base_path}{filename}.png"
    
    # Create plot
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(datetime_series, y_true, label='Actual', linewidth=2)
    ax.plot(datetime_series, y_pred, label='Predicted', color='red', linewidth=2)
    
    ax.set_xlabel(xlabel, fontsize=12)
    ax.set_ylabel(ylabel, fontsize=12)
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.legend(loc='upper right', fontsize=12)
    
    fig.autofmt_xdate()  # Auto-format date labels
    plt.tight_layout()
    
    # Save figure
    fig.savefig(save_path, dpi=300)
    plt.show()
    
    print(f"Plot saved to: {save_path}")

### Plot (Residual Dianostics)

In [47]:
def plot_and_save_residuals(datetime_series, y_true, y_pred, 
                            filename, 
                            base_path=base_path_plots):
    
     # Full save path
    save_path = f"{base_path}{filename}.png"

    # Calculate residuals
    residuals = y_true - y_pred

    # Create plot
    fig = plt.figure(figsize=(12, 8))
    gs = fig.add_gridspec(2, 2, height_ratios=[2, 1.5])

    # Residuals over time
    ax1 = fig.add_subplot(gs[0, :])
    ax1.plot(datetime_series, residuals, color='steelblue')
    ax1.axhline(0, linestyle='--', color='gray')
    ax1.set_ylabel("Residual (kW)")
    ax1.grid(True, linestyle='--', alpha=0.6)

    # ACF
    ax2 = fig.add_subplot(gs[1, 0])
    ax2.set_ylabel("ACF")
    plot_acf(residuals, lags=40, ax=ax2)

    # Histogram
    ax3 = fig.add_subplot(gs[1, 1])
    ax3.set_ylabel("Count")
    sns.histplot(residuals, bins=30, kde=True, color='salmon', ax=ax3)

    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.show()

### Plot (Prediction vs. Actual) for a Few Selected Weeks

In [48]:

def plot_and_save_weekly_predictions(datetime_series, y_true, y_pred, 
                                     filename, 
                                     base_path=base_path_plots,
                                     steps_per_week=96 * 7,  # 1 week of 15-min intervals
                                     weeks_to_plot=9):
    
    # Full save path
    save_path = f"{base_path}{filename}.png"
    
    total_steps = len(y_true)
    start_indices = np.linspace(0, total_steps - steps_per_week, weeks_to_plot, dtype=int)
    
    fig, axes = plt.subplots(3, 3, figsize=(15, 10), sharey=True)
    
    for idx, ax in enumerate(axes.flatten()):
        start = start_indices[idx]
        end = start + steps_per_week

        # Extract corresponding dates for titles
        start_date = datetime_series[start].strftime('%Y-%m-%d')
        end_date = datetime_series[end - 1].strftime('%Y-%m-%d')

        # Plot actual and predicted
        ax.plot(y_true[start:end], label='Actual', linewidth=2)
        ax.plot(y_pred[start:end], label='Predicted', color='red', linewidth=2)
        ax.set_title(f'Week: {start_date} to {end_date}', fontsize=10)
        ax.grid(True, linestyle='--', alpha=0.6)
        
        if idx % 3 == 0:
            ax.set_ylabel('Output (kW)', fontsize=10)
        if idx >= 6:
            ax.set_xlabel('Time Steps', fontsize=10)
    
    fig.legend(['Actual', 'Predicted'], loc='upper center', ncol=2, fontsize=12)
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    # Save figure
    fig.savefig(save_path, dpi=300)
    plt.show()
    
    print(f"Weekly predictions plot saved to: {save_path}")

## Model I: LightGBM

### Data Preparation

#### Define Target and Features

In [49]:
# Define features for LightGBM
features_lgbm = [
    'temperature',
    'is_holiday',
    'lag_1w', 'lag_1m', 'lag_1y',
    'hour_sin', 'hour_cos',
    'dayofweek_sin', 'dayofweek_cos',
    'month_sin', 'month_cos',
    'temp_roll_mean_1_day',
    'temp_roll_mean_7_day',
    'temp_roll_max_1_day',
    'temp_roll_min_1_day',
    'temp_roll_max_7_day',
    'temp_roll_min_7_day'
]

# Define target
target = 'output'

# Create input and output data for LightGBM
X_lgbm = data[features_lgbm]
y_lgbm = data[target]

#### Train-Test Split

In the case that all data should be imputed, more predictions are needed. Thus, a 50/50 split is done. Otherwise, an 80/20 split is done.

In [ ]:
# Function that decides the split based on the imputation strategy used
# 'test: An 80/20 split used for the test-time imputation strategy
# 'all': A 50/50 split used for the train-time imputation strategy
def determine_split(impute_type):
    if impute_type == 'all':
        split = 0.5
    elif impute_type == 'test':
        split = 0.8
    else:
        raise ValueError("Invalid impute_type. Choose 'all' or 'test'.")
    
    return split

In [51]:
# Train-test split
split = determine_split(impute_type='test')
split_idx = int(split * len(X_lgbm))

X_train_lgbm, X_test_lgbm = X_lgbm[:split_idx], X_lgbm[split_idx:]
y_train_lgbm, y_test_lgbm = y_lgbm[:split_idx], y_lgbm[split_idx:]

# Store corresponding dates for plotting
datetime_train_lgbm = y_lgbm.index[:split_idx]
datetime_test_lgbm = y_lgbm.index[split_idx:]

print("X_train shape:", X_train_lgbm.shape)
print("y_train shape:", y_train_lgbm.shape)

print("X_test shape:", X_test_lgbm.shape)
print("y_test shape:", y_test_lgbm.shape)

# Print date intervals for train and test sets
print(f"Train period: {datetime_train_lgbm[0].date()} to {datetime_train_lgbm[-1].date()}")
print(f"Test period:  {datetime_test_lgbm[0].date()} to {datetime_test_lgbm[-1].date()}")


X_train shape: (111129, 17)
y_train shape: (111129,)
X_test shape: (27783, 17)
y_test shape: (27783,)
Train period: 2020-12-31 to 2024-03-02
Test period:  2024-03-02 to 2024-12-16


### Hyper-Parameter Tuning

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Optuna Objective Function
def objective(trial):
    params = {
        'objective'         : 'regression',
        'metric'            : 'rmse',
        'boosting_type'     : 'gbdt',
        'verbosity'         : -1,
        'n_estimators'      : 1000,
        'num_leaves'        : trial.suggest_int('num_leaves', 8, 256, step=8),
        'max_depth'         : trial.suggest_int('max_depth', 3, 15),
        'min_child_samples' : trial.suggest_int('min_child_samples', 5, 100, step=5),
        'learning_rate'     : trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'subsample'         : trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree'  : trial.suggest_float('colsample_bytree', 0.5, 1),
        'max_bin'           : trial.suggest_int('max_bin', 50, 250, step=25),
        'reg_alpha'         : trial.suggest_float('reg_alpha', 1e-4, 1, log=True),
        'reg_lambda'        : trial.suggest_float('reg_lambda', 1e-4, 1, log=True)
    }

    model = LGBMRegressor(**params)

    model.fit(
        X_train_lgbm, y_train_lgbm,
        eval_set=[(X_test_lgbm, y_test_lgbm)],
        callbacks=[
            early_stopping(stopping_rounds=50)]
    )

    preds = model.predict(X_test_lgbm)
    rmse = root_mean_squared_error(y_test_lgbm, preds)
    return rmse

In [ ]:
# Run Optuna optimization
# optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Store best parameters
best_params = study.best_trial.params
print('Best RMSE:', study.best_trial.value)
print('Best Params:', best_params)

### Train/ Predict

In [ ]:
# Train final model with Best Parameters
model_lgbm = LGBMRegressor(**best_params, n_estimators=1000)

# Record start time
start_time = time.time()

# Train the model
model_lgbm.fit(
    X_train_lgbm, y_train_lgbm,
    eval_set=[(X_test_lgbm, y_test_lgbm)],
    callbacks=[
        early_stopping(stopping_rounds=50)
    ]
)

# Record end time
end_time = time.time()

In [ ]:
# # Save the model
# joblib.dump(model_lgbm, "/Users/Vageli/Desktop/DABN01/models/best_lgbm_model_all.pkl")

# # Load optimized model
# model_lgbm = joblib.load("/Users/Vageli/Desktop/DABN01/models/best_lgbm_model_all.pkl")

In [ ]:
# Calculate elapsed time in seconds
training_time_lgbm = end_time - start_time

# Format as minutes and seconds
training_time_minutes = training_time_lgbm / 60
print(f"Training Time: {training_time_lgbm:.2f} seconds ({training_time_minutes:.2f} minutes)")

In [ ]:
# Predict
y_pred_lgbm = model_lgbm.predict(X_test_lgbm)

results_lgbm = pd.DataFrame({
    'Prediction': y_pred_lgbm,
    'Outcome': y_test_lgbm.values
}, index=y_test_lgbm.index)

# Evaluaion metrics
metrics_lgbm = evaluate_regression_metrics(y_test_lgbm, y_pred_lgbm, label="LightGBM")


--- LightGBM Evaluation ---
Mean Absolute Error (MAE): 533.77 kW
Mean Absolute Percentage Error (MAPE): 10.69 %
Root Mean Squared Error (RMSE): 757.11 kW


### Plots

In [ ]:
# Plot interactive graph of predictions vs. actual output
plot_predictions(y_test_lgbm.values, y_pred_lgbm, dates=datetime_test_lgbm, title="LGBM Predictions on Test Set", label="LightGBM")

In [ ]:
# Calculate importance
importance = pd.Series(model_lgbm.feature_importances_, index=X_lgbm.columns)
importance = importance.sort_values(ascending=False)

# Plot importance
plt.figure(figsize=(10, 6))
importance.plot(kind='bar')
plt.title("Feature Importance")
plt.ylabel("Importance (Gain-based)")
plt.tight_layout()
plt.show()

In [ ]:
# Plot and store graph of predictions vs. actual output
plot_and_save_predictions(
    datetime_series=datetime_test_lgbm,
    y_true=y_test_lgbm.values,
    y_pred=y_pred_lgbm,
    filename="predictions_full(model=lgbm)"
)

In [ ]:
# Plot and store graph of residuals
plot_and_save_residuals(
    datetime_series=datetime_test_lgbm,
    y_true=y_test_lgbm.values,
    y_pred=y_pred_lgbm,
    filename="residuals(model=lgbm)"
)

residuals = y_test_lgbm - y_pred_lgbm
lb_test = acorr_ljungbox(residuals, lags=[10, 20, 30], return_df=True)
print("Ljung-Box Test Results:")
print(lb_test)

In [ ]:
# Plot and store graph of predictions vs. actual output for a few selected weeks
plot_and_save_weekly_predictions(
    datetime_series=datetime_test_lgbm,
    y_true=y_test_lgbm.values,
    y_pred=y_pred_lgbm,
    filename="predictions_week(model=lgbm)"
)

## LSTM

### Data imputation using LightGBM

In [28]:
# Parameters 
SEQUENCE_LEN = 96 * 7 # Number of past time steps to include in each input sequence to predict the next value
DELAY_LENGTH = 96 * 4 # Number of unavailable time steps that must be imputed

In [29]:
# Define features
feature_cols = [
    'output',
    'temperature',
    'is_holiday',
    'hour_sin', 'hour_cos',
    'dayofweek_sin', 'dayofweek_cos',
    'month_sin', 'month_cos'
]

# Convert DataFrame to NumPy Array 
data_lstm = data[feature_cols].values

In [30]:
# Create sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(seq_length, len(data)):
        X.append(data[i - seq_length:i])
        y.append(data[i][0])  # Target: 'output'
    return np.array(X), np.array(y)

X_lstm, y_lstm = create_sequences(data_lstm, SEQUENCE_LEN)
datetime_index_lstm = data.index[SEQUENCE_LEN:] # Adjust DateTime index to fit sequence-data

print(f"Sequence alignment check: {len(data_lstm) - len(X_lstm) == SEQUENCE_LEN}")

Sequence alignment check: True


In [31]:
def impute_and_split(X, y, y_pred_lgbm, delay_length, split_type='test'):
    X_imputed = X.copy()

    start_datetime = datetime_test_lgbm[0]
    start_id = datetime_index_lstm.get_loc(start_datetime)
    start_id_delay = start_id + delay_length

    assert len(y_pred_lgbm) == len(X[start_id_delay:]) + delay_length, "Predictions array too short for imputation."

    # Imputation
    for i in range(start_id_delay, len(X_imputed)):
        preds_slice = y_pred_lgbm[i - start_id_delay : i - start_id_delay + delay_length]
        X_imputed[i, -delay_length:, 0] = preds_slice

    # Split logic: 
    #   If 'test': Only imputes the test data
    #   If 'all': Splits data from where predictions start and performs an 80/20 train-test split
    if split_type == 'test':
        split_idx = start_id_delay

        X_train, X_test = X_imputed[:split_idx], X_imputed[split_idx:]
        y_train, y_test = y[:split_idx], y[split_idx:]

        datetime_train = datetime_index_lstm[:split_idx]
        datetime_test = datetime_index_lstm[split_idx:]

    elif split_type == 'all':
        X_imputed = X_imputed[start_id_delay:]
        y = y[start_id_delay:]
        datetime_filtered = datetime_index_lstm[start_id_delay:]

        split_idx = int(0.6 * len(X_imputed))

        X_train, X_test = X_imputed[:split_idx], X_imputed[split_idx:]
        y_train, y_test = y[:split_idx], y[split_idx:]

        datetime_train = datetime_filtered[:split_idx]
        datetime_test = datetime_filtered[split_idx:]

    else:
        raise ValueError("Invalid split_type. Choose 'all' or 'test'.")

    return X_train, X_test, y_train, y_test, datetime_train, datetime_test

In [33]:
X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm, datetime_train_lstm, datetime_test_lstm = impute_and_split(
    X_lstm,
    y_lstm,
    y_pred_lgbm,
    DELAY_LENGTH,
    split_type='all'
)

print("X_train shape:", X_train_lstm.shape)
print("y_train shape:", y_train_lstm.shape)

print("X_test shape:", X_test_lstm.shape)
print("y_test shape:", y_test_lstm.shape)

# Print date intervals for train and test sets
print(f"Train period: {datetime_train_lstm[0].date()} to {datetime_train_lstm[-1].date()}")
print(f"Test period:  {datetime_test_lstm[0].date()} to {datetime_test_lstm[-1].date()}")

X_train shape: (41443, 672, 9)
y_train shape: (41443,)
X_test shape: (27629, 672, 9)
y_test shape: (27629,)
Train period: 2022-12-28 to 2024-03-04
Test period:  2024-03-04 to 2024-12-16


### Train/ Predict

In [ ]:
# Create normalization layer
norm_layer = layers.Normalization()

# Adapt it to your sequence training data (shape: (samples, timesteps, features))
norm_layer.adapt(X_train_lstm)

In [ ]:
# Scale target variable 
scaler_y = MinMaxScaler()
y_train_lstm_scaled = scaler_y.fit_transform(y_train_lstm.reshape(-1, 1))

In [ ]:
# Model parameters
input_shape = X_train_lstm.shape[1:]  # (timesteps, features)
dropout_rate = 0.2
lstm_units = 64
batch_size = 64
epochs = 5

# Define the model
model_lstm = models.Sequential([
    Input(shape=input_shape),

    norm_layer,

    layers.LSTM(lstm_units, return_sequences=True),
    layers.Dropout(dropout_rate),

    layers.LSTM(lstm_units),
    layers.Dropout(dropout_rate),

    layers.Dense(1)
])

model_lstm.compile(optimizer='adam', loss='mse')

# Early stopping
early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model_lstm.summary()

In [ ]:
# Record start time
start_time = time.time()

# Train the model
history = model_lstm.fit(
    X_train_lstm, y_train_lstm,
    validation_split=0.2,
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[early_stop],
    verbose=1
)

# Record end time
end_time = time.time()

In [ ]:
# Calculate elapsed time
training_time_lstm = end_time - start_time

# Format as minutes and seconds
training_time_minutes = training_time_lstm / 60
print(f"Training Time: {training_time_lstm:.2f} seconds ({training_time_minutes:.2f} minutes)")

In [ ]:
import matplotlib.pyplot as plt

def plot_training_history(history, title="Training vs Validation Loss"):
    plt.figure(figsize=(8, 5))
    plt.plot(history.history['loss'], label='Training Loss', linewidth=2)
    plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2, linestyle='--')
    
    plt.title(title, fontsize=14)
    plt.xlabel('Epochs', fontsize=12)
    plt.ylabel('Loss (MSE)', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend(fontsize=12)
    plt.tight_layout()
    plt.show()

# Plot the loss curves
plot_training_history(history)

In [ ]:
# Predict and inverse transform
y_pred_lstm_scaled = model_lstm.predict(X_test_lstm).flatten()
y_pred_lstm = scaler_y.inverse_transform(y_pred_lstm_scaled.reshape(-1, 1)).flatten()

results_lgbm = pd.DataFrame({
    'Prediction': y_pred_lstm,
    'Outcome': y_test_lstm
}, index=datetime_test_lstm)

# Evaluaion metrics
metrics = evaluate_regression_metrics(y_test_lstm, y_pred_lstm, label="LSTM")

### Plots

In [ ]:
# Plot interactive graph of predictions vs. actual output
plot_predictions(y_test_lstm, y_pred_lstm, dates=datetime_test_lstm, title="LSTM Predictions on Test Set", label="LSTM")

In [ ]:
# Plot and store graph of predictions vs. actual output
plot_and_save_predictions(
    datetime_series=datetime_test_lstm,
    y_true=y_test_lstm,
    y_pred=y_pred_lstm,
    filename="predictions_full(model=lstm,data=imput,seq=672,layer=2)"
)

In [ ]:
# Plot and store graph of residuals
plot_and_save_residuals(
    datetime_series=datetime_test_lstm,
    y_true=y_test_lstm,
    y_pred=y_pred_lstm,
    filename="residuals(model=lstm,data=imput,seq=672,layer=2)"
)

In [ ]:
# Plot and store graph of predictions vs. actual output for a few selected weeks
plot_and_save_weekly_predictions(
    datetime_series=datetime_test_lstm,
    y_true=y_test_lstm,
    y_pred=y_pred_lstm,
    filename="predictions_week(model=lstm,data=imput,seq=672,layer=2)"
)

### Model III: Naive Seasonal Model

In [30]:
import pandas as pd

# Set the lag period (1 week = 7 * 96 quarter-hours)
lag_steps = 7 * 96

# Test period start datetime (timezone-aware)
test_start = pd.Timestamp("2024-03-06 13:15:00+00:00")

# Create shifted predictions based on last week's output
seasonal_naive_pred = data['output'].shift(lag_steps)

# Create new DataFrame for results (index remains datetime)
seasonal_naive_results = pd.DataFrame({
    'true_output': data['output'],
    'seasonal_naive_pred': seasonal_naive_pred
})

# Filter to start from test period
seasonal_naive_results = seasonal_naive_results.loc[test_start:]

# Drop rows with NaN predictions
seasonal_naive_results = seasonal_naive_results.dropna()

In [31]:
# Save to CSV, including the datetime index
save_path = "/Users/Vageli/Desktop/DABN01/data/seasonal_naive_predictions.csv"
seasonal_naive_results.to_csv(save_path)